# Wrangling OpenStreetMap Data by Rohit Vinnakota

## 1. Parsing the file

In [26]:
import xml.etree.ElementTree as ET
from collections import defaultdict
from bs4 import BeautifulSoup
import pprint
import csv
import codecs
import cerberus
import schema
import re

FILEPATH = "edmonton_canada.osm"


# Helper functions for parsing and wrangling


#Returns true if the element is a street name
def is_street_name(elem):
    if (elem.attrib['k'] == "addr:street"):
        return True

# Counts and returns the different number of tags and their types
def count_tags(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    dict1={}
    dict1[root.tag] = 1
    dup_list = []
    for a in root.findall('.//'):
        if a.tag not in dup_list:
            dup_list.append(a.tag)
            dict1[a.tag] = 1
    else:
        dict1[a.tag]+= 1

    return dict1


#Returns the number of users
def list_of_users(filename):
    users = set()
    user_cont = {}
    for _, elem in ET.iterparse(filename):
        users.add(elem.attrib.get('user'))
        if elem.attrib.get('user') in user_cont:
            user_cont[elem.attrib.get('user')]+=1
        else:
            user_cont[elem.attrib.get('user')] = 1
    del user_cont[None] #Some elements have an empty user value, I delete all the "None" users in this case
    return users,user_cont

#Returns true if the element is a zipcode
def is_zipcode(elem):
    if (elem.attrib['k'] == "addr:postcode"):
        return True
 
#Audits non existent names in our final street list 
def audit_street(street_types,name): 
    lastTwoWords = name.split()[-2:] # http://stackoverflow.com/questions/13482586/how-can-i-tell-python-to-take-the-last-2-words-from-an-email-that-is-in-a-list-l
    if len(lastTwoWords) == 2:
        for i in range(2):
            if (lastTwoWords[i] not in final_street_list):
                street_types[lastTwoWords[i]].add(name)

#Audits incorrect zipcodes
def audit_zipcode(zipcode,wrong_codes):
    if zipcode[:1] != 'T' or len(zipcode) != 6:
        wrong_codes.append(zipcode)


print(count_tags(FILEPATH))


{'node': 1, 'nd': 1, 'bounds': 1, 'member': 1, 'tag': 2, 'osm': 1, 'way': 1, 'relation': 1}


## 2. Using site scraping to obtain a list of valid street names

Rather than typing up all the street names, I will use beautiful soup on the attached html file(URL: http://gimme-shelter.com/term-2/steet-types-designations-abbreviations-50006/) to obtain a list of all the street names

In [13]:
soup1 = BeautifulSoup(open("Streets.html"),"lxml")

In [14]:
street_list = []

tables = (soup1.findAll('tbody'))         #returns list of all the tables. 
                                      #there is only one table in this case
    
row_list = tables[0].findAll('tr')
for row in row_list:
    name_list = row.findAll('td')
    for name in name_list:
        street = name.string
        street_list.append(street)
            
print(street_list)

[u'Street Type', u'Abbreviation (Mail)', u'Abbreviation (Edmonton)', None, None, None, u'Abbey', u'ABBEY', u'\u2013', u'Acres', u'ACRES', u'AC', u'Alley or All\xe9e', u'ALLEY or ALL\xc9E', u'AL', u'Autoroute', u'AUT', u'\u2013', u'Avenue', u'AVE or AV', u'AV', u'Bay', u'BAY', u'BA', u'Beach', u'BEACH', u'BE', u'Bend', u'BEND', u'BN', u'Boulevard', u'BLVD or BOUL', u'BV', u'By-pass', u'BYPASS', u'\u2013', u'Byway', u'BYWAY', u'\u2013', u'Campus', u'CAMPUS', u'\u2013', u'Cape', u'CAPE', u'CA', u'Carr\xe9', u'CAR', u'\u2013', u'Carrefour', u'CARREF', u'\u2013', u'Castle', u'\u2013', u'CS', u'Centre', u'CTR or C', u'CE', u'Cercle', u'CERCLE', u'\u2013', u'Chase', u'CHASE', u'\u2013', u'Chemin', u'CH', u'\u2013', u'Circle', u'CIR', u'CI', u'Circuit', u'CIRCT', u'\u2013', u'Close', u'CLOSE', u'CL', u'Common', u'COMMON', u'CM', u'Concession', u'CONC', u'\u2013', u'Corners', u'CRNRS', u'\u2013', u'C\xf4te', u'C\xd4TE', u'\u2013', u'Cour', u'COUR', u'\u2013', u'Cours', u'COURS', u'\u2013', u'Co

In [15]:
street_list = street_list[6:]
final_street_list = []
for i in range(len(street_list)):
    if i % 3 == 0:
        final_street_list.append(street_list[i])
print(final_street_list)
#Split the or's

[u'Abbey', u'Acres', u'Alley or All\xe9e', u'Autoroute', u'Avenue', u'Bay', u'Beach', u'Bend', u'Boulevard', u'By-pass', u'Byway', u'Campus', u'Cape', u'Carr\xe9', u'Carrefour', u'Castle', u'Centre', u'Cercle', u'Chase', u'Chemin', u'Circle', u'Circuit', u'Close', u'Common', u'Concession', u'Corners', u'C\xf4te', u'Cour', u'Cours', u'Court', u'Cove', u'Crescent or Croissant', u'Crest', u'Cross', u'Crossing', u'Cul-de-sac', u'Dale', u'Dell', u'Diversion', u'Downs', u'Drive', u'\xc9changeur', u'End', u'Esplanade', u'Estates', u'Expressway', u'Extension', u'Farm', u'Field', u'Forest', u'Freeway', u'Front', u'Gardens', u'Gate', u'Gateway', u'Glade', u'Glen', u'Green', u'Grounds', u'Grove', u'Hall', u'Harbour', u'Haven', u'Heath', u'Heights', u'Highlands', u'Highway', u'Hill or Hills', u'Hollow', u'\xcele', u'Impasse', u'Inlet', u'Island', u'Keep', u'Key', u'Knoll', u'Lake', u'Landing', u'Lane', u'Limits', u'Line', u'Link', u'Lookout', u'Loop', u'Mall', u'Manor', u'Maze', u'Meadow or Meadow

In [16]:
for item in final_street_list:
    if item.find(" or") != -1:
        N = item.split()
        final_street_list.append(N[0])
        final_street_list.append(N[2])
        final_street_list.remove(item)
        
print(final_street_list)
    

[u'Abbey', u'Acres', u'Autoroute', u'Avenue', u'Bay', u'Beach', u'Bend', u'Boulevard', u'By-pass', u'Byway', u'Campus', u'Cape', u'Carr\xe9', u'Carrefour', u'Castle', u'Centre', u'Cercle', u'Chase', u'Chemin', u'Circle', u'Circuit', u'Close', u'Common', u'Concession', u'Corners', u'C\xf4te', u'Cour', u'Cours', u'Court', u'Cove', u'Crest', u'Cross', u'Crossing', u'Cul-de-sac', u'Dale', u'Dell', u'Diversion', u'Downs', u'Drive', u'\xc9changeur', u'End', u'Esplanade', u'Estates', u'Expressway', u'Extension', u'Farm', u'Field', u'Forest', u'Freeway', u'Front', u'Gardens', u'Gate', u'Gateway', u'Glade', u'Glen', u'Green', u'Grounds', u'Grove', u'Hall', u'Harbour', u'Haven', u'Heath', u'Heights', u'Highlands', u'Highway', u'Hollow', u'\xcele', u'Impasse', u'Inlet', u'Island', u'Keep', u'Key', u'Knoll', u'Lake', u'Landing', u'Lane', u'Limits', u'Line', u'Link', u'Lookout', u'Loop', u'Mall', u'Manor', u'Maze', u'Mews', u'Mont\xe9e', u'Moor', u'Mount', u'Mountain', u'Not Applicable', u'Oaks', u

## 3. Basic querying using python libraries

After some cleaning , we can construct an appropriate mapping dictionary. But first, let us get some insights on the uncleaned data. 

In [27]:

x,y = list_of_users(FILEPATH)
print(len(x))
print("\n")

619




The number of users who contributed to the website for the map area chosen 

In [28]:

# http://stackoverflow.com/questions/26871866/print-highest-value-in-dict-with-key

maximum = max(y, key = y.get)  # Just use 'min' instead of 'max' for minimum.
print(maximum, y[maximum])

('edmontongeo', 2407878)


This is the user with the most contributions. It seems to be a bot or a company which would be appropriate. 

## 4. Updating incorrect street names and auditing zipcodes

In [19]:
street_types = defaultdict(set) #Audit whole street type instead of last word


for event, elem in ET.iterparse(FILEPATH, events=("start",)):# Change to iterate over last 2 elements instead of last one
    if elem.tag == "way" or elem.tag == "node":
        for tag in elem.iter("tag"):
            if (is_street_name(tag)):
                audit_street(street_types,tag.attrib['v'])
pprint.pprint(dict(street_types))




{'1': set(['1 Avenue', '1 Street']),
 '10': set(['10 Street', '50322 Range Road 10', 'Range Road  10']),
 '100': set(['100 Avenue', '100 Street', 'Sturgeon 100 Crescent']),
 '100A': set(['100A Avenue', '100A Street']),
 '101': set(['101 Avenue', '101 Street', '101 street']),
 '101A': set(['101A Street']),
 '102': set(['102 Avenue', '102 Street']),
 '103': set(['103 Avenue', '103 Street']),
 '103A': set(['103A Avenue', '103A Street']),
 '104': set(['104 Avenue', '104 Street']),
 '105': set(['105 Avenue', '105 Street']),
 '106': set(['106 Avenue', '106 Street']),
 '106A': set(['106A Avenue']),
 '107': set(['107 Avenue', '107 St', '107 Street', '107 street']),
 '107A': set(['107A Avenue']),
 '108': set(['108 Avenue', '108 Street']),
 '108A': set(['108A Avenue', '108A Street']),
 '109': set(['109 Avenue', '109 Street', '109 street']),
 '109A': set(['109A Street']),
 '11': set(['11 Avenue', 'Range Road  11']),
 '110': set(['110 Avenue', '110 Street']),
 '110A': set(['110A Street']),
 '111':

Apart from numerical highway names and unique street names, there are a surprisingly low amount of inconsistent street types in the dataset. 

In [20]:
mapping = {'Northwest': 'NW', 'Northeast': 'NE', 'Southwest': 'SW', 'Southeast': 'SE', 
           'North-west': 'NW', 'North-east': 'NE', 'South-west': 'SW', 'South-east': 'SE','St':'Street',
           'Ave':'Avenue','Blvd':'Boulevard'}

for event, elem in ET.iterparse(FILEPATH, events=("start",)):# Change to iterate over last 2 elements instead of last one
    if elem.tag == "way":
        for tag in elem.iter("tag"):
            if (is_street_name(tag)):
                print(tag.attrib['v'])
    
#The update mapping function can be contextually created for auditing street names after viewing the incorrect street names 
def update_street(street_name,mapping):
    street_words = street_name.split()
    
    if street_words[len(street_words)-1] != ("North-east" or "North-west" or "South-east"
                                             or "South-west" or "Northeast" or "Northwest" or "Southeast"
                                             or "Southwest"):
        if street_words[len(street_words)-1] in mapping:
            street_words[len(street_words)-1] = mapping[street_words[len(street_words)-1]]
            
    for i in range(len(street_words)):
        if street_words[i] in mapping:
            street_words[i] = mapping[street_words[i]]
    street_name = " ".join(street_words)
    return street_name

for x, ways in street_types.iteritems():
    for name in ways:
        better_name = update_street(name, mapping)
        print name, "=>", better_name

109 street
66 Street NW
82 Street NW
50 Street NW
102A Avenue NW
102 Street NW
142 Street North-west
130 Street NW
111 Avenue NW
Gateway Boulevard NW
144 Avenue NW
23 Avenue NW
107 Street NW
107 Street NW
Wye Road
Wye Road
Wye Road
Wye Road
81 Street North-west
112 Avenue NW
81 Ave
109 Street North-west
82 Avenue North-west
Saskatchewan Drive NW
Whyte Avenue NW
84 Avenue NW
106 Street NW
106 Avenue NW
107 Street NW
Kingsway NW
101 Street NW
66 Street NW
23 Avenue NW
21 Street NW
100 Avenue
Gateway Blvd
Gateway Boulevard North-west
104th
101 Street NW
50 Street NW
Parsons Road
142 Street NW
80 Avenue NW
11A Avenue NW
48 Street NW
85 Street
23 Avenue NW
111 Avenue NW
Calgary Trail South
17 Avenue NW
80 Street NW
132 St.
103A Avenue
106 Street NW
111 St NW
Hebert Road
182 Street NW
86
115 Street NW
54 Avenue NW
51 Avenue NW
106 Street NW
36A Avenue NW
43 Avenue North-west
Fairway Drive NW
45 Avenue NW
Ellerslie Road SW
148 Street NW
56 Avenue NW
114 Street NW
Jasper Court South
Haven Aven

Zipcodes in Edmonton must have the following format:

1)The first 3 letters must be T5A
2)The next character must be 0,1, or 2
3)The next character must be an alphabet
4)The next character must be a number between 1-9
5)Overall the zipcode's length must be 6

We can check if there are any zipcodes that violate these rules and audit them accordingly. 

In [21]:
"""Zipcodes in Edmonton must have the following format:
The first 3 letters must be T5A and the length of the zipcode must be 6.
We can check if there are any zipcodes that violate these rules and audit them accordingly.
Due to the low number of incorrect zipcodes, I choose to remove the incorrect zipcodes"""


wrong_codes = []

    
for event, elem in ET.iterparse(FILEPATH, events=("start",)):# Change to iterate over last 2 elements instead of last one
    if elem.tag == "way":
        for tag in elem.iter("tag"):
            if (is_zipcode(tag)):
                tag.attrib['v'] = tag.attrib['v'].replace(" ", "")
                audit_zipcode(tag.attrib['v'],wrong_codes)
                if tag in wrong_codes:
                    tag.attrib['v'] = None
                    

                

## 5. Converting our data to prepare for insertion into an SQL database

In [22]:
#Conversion to SQL schema taken from "Preparing for Database - SQL" from the Case Study:Open street map Module



OSM_PATH = FILEPATH

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for item in NODE_FIELDS:
            if element.attrib[item] is not None:
                node_attribs[item] = element.attrib[item] #Gets the value for the key
        for child in element.iter("tag"): #all children who's name is tag
            if problem_chars.match(child.attrib['k']): #ignore the tag
                continue
            tag_to_insert = {} #node tagged
            tag_to_insert['id'] = node_attribs['id']     
            key_check = child.attrib['k']
            if child.attrib['k'] == "addr:street":
                child.attrib['v'] = update_street(child.attrib['v'],mapping)
            if child.attrib['k'] == "addr:postcode":
                filler = []
                child.attrib['v'] = child.attrib['v'].replace(" ", "")
                child.attrib['v'] = audit_zipcode(child.attrib['v'],filler)
            if ':' in key_check:
                ind = child.attrib['k'].index(":")
                tag_to_insert['key'] = child.attrib['k'][ind+1:]
                tag_to_insert['value'] = child.attrib['v']
                tag_to_insert['type'] = child.attrib['k'][:ind]
            else:
                tag_to_insert['key'] = child.attrib['k']
                tag_to_insert['type'] = "regular"
            tags.append(tag_to_insert)
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for item in WAY_FIELDS:
            if element.attrib[item] is not None:
                way_attribs[item] = element.attrib[item]
        
        for child in element.iter("tag"):
            if problem_chars.match(child.attrib['k']): #ignore the tag
                continue
            else:
                tag_to_insert = {} #node tagged
                tag_to_insert['id'] = way_attribs['id']     
                key_check = child.attrib['k']
                if child.attrib['k'] == "addr:street":
                    child.attrib['v'] = update_street(child.attrib['v'],mapping)
                if child.attrib['k'] == "addr:postcode":
                    filler = []
                    child.attrib['v'] = child.attrib['v'].replace(" ", "")
                    child.attrib['v'] = audit_zipcode(child.attrib['v'],filler)
                if ':' in key_check:
                    ind = child.attrib['k'].index(":")
                    tag_to_insert['key'] = child.attrib['k'][ind+1:]
                    tag_to_insert['value'] = child.attrib['v']
                    tag_to_insert['type'] = child.attrib['k'][:ind]
                else:
                    tag_to_insert['key'] = child.attrib['k']
                    tag_to_insert['type'] = "regular"
                tags.append(tag_to_insert)
        c = 0
        for child in element.iter():
            if child.tag == "nd":
                nd ={}
                nd['id'] = way_attribs['id']
                nd['node_id'] = child.attrib['ref']
                nd['position'] = c
                c+=1
                way_nodes.append(nd)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



# ================================================== #
#               Helper Functions                     #
# ================================================== #
SCHEMA = schema.schema
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )



class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])



#I am only using process_map to validate the data in the sample_testing file since it is too slow for a larger dataset
process_map(OSM_PATH, validate=False)


## 6. Further insights through querying 

I will now create a database using sqlite3 and the csv files and query the database to obtain more insights. First, we can check the number of user who contributed to the code. It should be around 619. 

### FILE SIZES

edmonton_canada.osm => 784 MB

nodes.csv => 312 MB
nodes_tags.csv => 18.95 MB

ways.csv => 28.05 MB

ways_tags.csv => 102.4 MB

ways_nodes.csv => 25.6 MB

p3main.db(the sql database) => 581 MB

In [22]:
import sqlite3

#Setting up cursor and querying the number of unique users
conn = sqlite3.connect('p3final.db')
conn.text_factory = str
c = conn.cursor()

query = ("SELECT count(DISTINCT user) FROM (SELECT user FROM ways UNION ALL SELECT user FROM nodes);")
c.execute(query)
print(c.fetchall()[0][0])

614


In [24]:
#The dataset is massive. The number of entries in each table are: 

query = ("SELECT count(*) FROM nodes");
c.execute(query)
print("nodes: "+ str(c.fetchall()[0][0]))
query = ("SELECT count(*) FROM ways");
c.execute(query)
print("ways: "+ str(c.fetchall()[0][0]))
query = ("SELECT count(*) FROM nodes_tags")
c.execute(query)
print("nodes_tags: " + str(c.fetchall()[0][0]))
query = ("SELECT count(*) FROM ways_tags")
c.execute(query)
print("ways_tags: "+ str(c.fetchall()[0][0]))
query = ("SELECT count(*) FROM ways_nodws")
c.execute(query)
print("ways_nodes: "+ str(c.fetchall()[0][0]))

nodes: 3698501
ways: 465338
nodes_tags: 557266
ways_tags: 871375
ways_nodes: 4202209


In [9]:
#Now querying to find the user with the most submissions

query = ("SELECT user,count(user) FROM nodes GROUP BY user UNION SELECT user,count(user) FROM ways GROUP BY user ORDER BY count(user) DESC;")
c.execute(query)
print(c.fetchall()[0])

('edmontongeo', 2119823)


It is not surprising that edmontongeo remains the user with the highest submissions

In [10]:
query =("SELECT distinct value,count(value) FROM nodes_tags WHERE key LIKE '%city' GROUP BY value UNION SELECT distinct value,count(value) FROM ways_tags WHERE key LIKE '%city' GROUP BY value ORDER BY count(value) DESC;")
c.execute(query)
print c.fetchall()

[('Edmonton', 75598), ('Parkland County', 5552), ('Strathcona County', 5289), ('Sturgeon County', 4910), ('St. Albert', 4540), ('Leduc County', 3876), ('Westlock County', 2558), ('Leduc', 2465), ('Spruce Grove', 2066), ('Fort Saskatchewan', 1903), ('Lac Ste. Anne County', 1595), ('Wetaskiwin', 1425), ('Stony Plain', 1422), ('Thorhild No. 7, County of', 1403), ('Wetaskiwin No. 10, County of', 1281), ('Beaumont', 1264), ('Barrhead No. 11, County of', 1126), ('Morinville', 911), ('Edmonton', 845), ('Barrhead', 711), ('Westlock', 632), ('Devon', 597), ('Redwater', 400), ('Gibbons', 334), ('Camrose County', 333), ('Millet', 292), ('Sherwood Park', 281), ('Alberta Beach', 266), ('Thorsby', 248), ('Calmar', 247), ('Nisku', 239), ('Legal', 230), ('Bon Accord', 222), ('Thorhild', 214), ('Warburg', 206), ('Onoway', 192), ('Cardiff', 152), ('Wabamun', 148), ('Breton', 126), ('Clyde', 126), ('Hay Lakes', 82), ('Brazeau County', 77), ('Spring Lake', 76), ('Val Quentin', 72), ('New Sarepta', 64), ('

The data appears very inconsistent with multiple "Edmonton" city values. This is one of the major problems with user submitted data. In addition to this, we can conclude that the map area used is not only that for edmonton but for surrounding towns and counties as well. 

In [32]:
query = ("SELECT key,count(*) FROM nodes_tags GROUP BY key ORDER BY count(*) DESC")
c.execute(query)
print(c.fetchall())

[('source', 130584), ('housenumber', 126644), ('street', 126642), ('city', 126521), ('highway', 12398), ('power', 10497), ('name', 9490), ('amenity', 2120), ('fixme', 1135), ('natural', 781), ('railway', 777), ('barrier', 677), ('created_by', 648), ('shop', 549), ('place', 528), ('type', 503), ('website', 448), ('cuisine', 342), ('en', 322), ('operator', 320), ('building', 319), ('postcode', 242), ('species', 221), ('man_made', 203), ('phone', 200), ('network', 184), ('ref', 177), ('leisure', 170), ('tourism', 168), ('wikipedia', 148), ('crossing', 136), ('traffic_signals', 136), ('province', 110), ('wikidata', 105), ('note', 98), ('population', 92), ('entrance', 88), ('opening_hours', 61), ('sport', 61), ('religion', 58), ('traffic_calming', 58), ('country', 53), ('description', 51), ('covered', 50), ('is_in', 50), ('bicycle_parking', 49), ('waterway', 49), ('aeroway', 46), ('foot', 45), ('adm1', 44), ('cc1', 44), ('comment', 44), ('dms_lat', 44), ('dms_long', 44), ('dsg', 44), ('fc',

This query reveals a major problem. Most of these values for the corresponding keys are empty. 

A lot of random querying such as (SELECT * FROM nodes_tags WHERE key= 'amenity') produced null results in the values column. This suggests further cleaning is required to obtain reasonable insights. Almost all nodes_tags have a key but no value. This shows a lack of full utilization of the OSM featureset or the existence of problematic characters. A major reason may be that Edmonton is not an extremely dense or populus city like New York or San Francisco thus has a lower userbase constantly updating data. Some suggestions for improvement include a unified data entry scheme on the OSM website, where unnecesscary tags are not created if the user does not fill in the value. This can work in a format similar to google docs, where the user fill out their pertinent contribution in a form(name of the place, type of place, address, zipcode, etc) and the data is inserted through a backend process which can also act as a auto-correcter to prevent the user from entering nonsensical data. 

As it stands, there remains no initiative to have a unified format for the data. This can be fixed by automating a lot of self-correction and user entry processes to make the data that does pass through cleaner than raw user submitted data. 

## Sources

http://gimme-shelter.com/term-2/steet-types-designations-abbreviations-50006/

https://www.crummy.com/software/BeautifulSoup/bs3/documentation.html#The%20basic%20find%20method:%20findAll(name,%20attrs,%20recursive,%20text,%20limit,%20**kwargs)

http://stackoverflow.com/questions/13482586/how-can-i-tell-python-to-take-the-last-2-words-from-an-email-that-is-in-a-list-l

http://stackoverflow.com/questions/26871866/print-highest-value-in-dict-with-key

https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md

All relevent coursework in Udacity Data Analyst Nanodegree(especially the Case:Study OSM and XML module)